In [2]:
import numpy as np
import pandas as pd
from doubleml.datasets import fetch_bonus
from doubleml import DoubleMLData
import statsmodels.api as sm
from sklearn.base import clone
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LassoCV
from doubleml import DoubleMLPLR   
from doubleml import DoubleMLPLR  ,DoubleMLPLIV         
from prettytable import PrettyTable
from PIL import Image, ImageDraw, ImageFont
from sklearn.linear_model import LinearRegression, LogisticRegression
from doubleml.datasets import fetch_401K

In [3]:
df = pd.read_csv('/Users/pranjal/Desktop/Causal-Inference/data/fertil1.csv')
df.head()

,year,educ,meduc,feduc,age,kids,black,east,northcen,west,...,y80,y82,y84,agesq,y74educ,y76educ,y78educ,y80educ,y82educ,y84educ
0,72,12,8,8,48,4,0,0,1,0,...,0,0,0,2304,0,0,0,0,0,0
1,72,17,8,18,46,3,0,0,0,0,...,0,0,0,2116,0,0,0,0,0,0
2,72,12,7,8,53,2,0,0,1,0,...,0,0,0,2809,0,0,0,0,0,0
3,72,12,12,10,42,2,0,0,1,0,...,0,0,0,1764,0,0,0,0,0,0
4,72,12,3,8,51,2,0,0,0,0,...,0,0,0,2601,0,0,0,0,0,0


In [4]:
np.random.seed(1)
data = fetch_401K(return_type='DataFrame')
df.head(5)
y = df['kids']
d = df['educ']
x = df[['age', 'black', 'east', 'northcen', 'west', 'farm', 'othrural', 'town', 'smcity', 'y74', 'y76', 'y78' ,'y80','y82' ,'y84']]
z = df[['meduc', 'feduc']]

l = RandomForestRegressor(n_estimators = 500, max_features = 'sqrt', max_depth= 5)
g = clone(l)
m = RandomForestClassifier(n_estimators = 500, max_features = 'sqrt', max_depth= 5)
r = RandomForestRegressor(n_estimators = 500, max_features = 'sqrt', max_depth= 5)

def score(y, d, l_hat, m_hat, g_hat, smpls):
    u_hat = y - g_hat
    psi_a = -np.multiply(d, d)
    psi_b = np.multiply(d, u_hat)
    return psi_a, psi_b

print(y.shape, x.shape, d.shape, z.shape)

(1129,) (1129, 15) (1129,) (1129, 2)


In [12]:
import statsmodels.api as sm
OLS = sm.OLS(y, sm.add_constant(np.c_[d, x])).fit()

In [13]:
from statsmodels.sandbox.regression.gmm import IV2SLS
endog = df['kids']
exog = df[[ 'educ','age', 'black', 'east', 'northcen', 'west', 'farm', 'othrural', 'town', 'smcity', 'y74', 'y76', 'y78' ,'y80','y82' ,'y84']]
instr = df[['age', 'black', 'east', 'northcen', 'west', 'farm', 'othrural', 'town', 'smcity', 'y74', 'y76', 'y78' ,'y80','y82' ,'y84', 'meduc','feduc']]
exog_constant = sm.add_constant(exog)
instr_constant = sm.add_constant(instr)
IV2SLS = IV2SLS(endog, exog_constant, instrument = instr_constant).fit()
IV2SLS.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                          IV2SLS Regression Results                           
==============================================================================
Dep. Variable:                   kids   R-squared:                       0.117
Model:                         IV2SLS   Adj. R-squared:                  0.105
Method:                     Two Stage   F-statistic:                     7.170
                        Least Squares   Prob (F-statistic):           4.76e-16
Date:                Tue, 13 Dec 2022                                         
Time:                        12:46:58                                         
No. Observations:                1129                                         
Df Residuals:                    1112                                         
Df Model:                          16                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.7901      0.670      5.653      0.000       2.475       5.106
educ          -0.1565      0.039     -3.976      0.000      -0.234      -0.079
age            0.0187      0.008      2.273      0.023       0.003       0.035
black          1.0488      0.175      6.007      0.000       0.706       1.391
east           0.2321      0.135      1.724      0.085      -0.032       0.496
northcen       0.3622      0.123      2.951      0.003       0.121       0.603
west           0.1817      0.169      1.078      0.281      -0.149       0.512
farm          -0.0792      0.152     -0.520      0.603      -0.378       0.219
othrural      -0.1938      0.183     -1.061      0.289      -0.552       0.165
town           0.0862      0.125      0.688      0.492      -0.160       0.332
smcity         0.2282      0.161      1.415      0.157      -0.088       0.545
y74            0.2442      0.174      1.405      0.160      -0.097       0.585
y76           -0.1376      0.180     -0.765      0.445      -0.491       0.215
y78           -0.0955      0.183     -0.521      0.603      -0.455       0.264
y80           -0.0718      0.186     -0.387      0.699      -0.436       0.293
y82           -0.5257      0.178     -2.961      0.003      -0.874      -0.177
y84           -0.5638      0.179     -3.154      0.002      -0.914      -0.213
==============================================================================
Omnibus:                        8.955   Durbin-Watson:                   2.009
Prob(Omnibus):                  0.011   Jarque-Bera (JB):                9.089
Skew:                           0.219   Prob(JB):                       0.0106
Kurtosis:                       2.959   Cond. No.                         482.
==============================================================================
"""

In [6]:
data = DoubleMLData(df.sample(int(df.shape[0]*0.8)), 'kids', 'educ', z_cols=['meduc', 'feduc'])
SML = DoubleMLPLIV(data, l,m,g,r, n_folds=1,apply_cross_fitting=False,score=score)
#SML.fit();

/usr/local/lib/python3.10/site-packages/doubleml/double_ml.py:953: UserWarning: Learner provided for ml_m is probably invalid: RandomForestClassifier(max_depth=5, n_estimators=500) is (probably) no regressor.
  warnings.warn(warn_msg_prefix + f'{str(learner)} is (probably) no regressor.')


In [8]:
data = DoubleMLData(df.sample(int(df.shape[0]*1)), 'kids', 'educ', z_cols=['meduc', 'feduc']) 
OML = DoubleMLPLIV(data,l,m,g,r, n_folds=1,apply_cross_fitting=False)
#OML.fit();

/usr/local/lib/python3.10/site-packages/doubleml/double_ml.py:953: UserWarning: Learner provided for ml_m is probably invalid: RandomForestClassifier(max_depth=5, n_estimators=500) is (probably) no regressor.
  warnings.warn(warn_msg_prefix + f'{str(learner)} is (probably) no regressor.')
/usr/local/lib/python3.10/site-packages/doubleml/double_ml_pliv.py:153: UserWarning: A learner ml_g has been provided for score = "partialling out" but will be ignored. "A learner ml_g is not required for estimation.
  warnings.warn(('A learner ml_g has been provided for score = "partialling out" but will be ignored. "'


In [9]:
data = DoubleMLData(df.sample(int(df.shape[0]*1)), 'kids', 'educ', z_cols=['meduc', 'feduc'])
DML = DoubleMLPLIV(data, l,m,g,r, n_folds=5,apply_cross_fitting=True)
DML.fit();

/usr/local/lib/python3.10/site-packages/doubleml/double_ml.py:953: UserWarning: Learner provided for ml_m is probably invalid: RandomForestClassifier(max_depth=5, n_estimators=500) is (probably) no regressor.
  warnings.warn(warn_msg_prefix + f'{str(learner)} is (probably) no regressor.')
/usr/local/lib/python3.10/site-packages/doubleml/double_ml_pliv.py:153: UserWarning: A learner ml_g has been provided for score = "partialling out" but will be ignored. "A learner ml_g is not required for estimation.
  warnings.warn(('A learner ml_g has been provided for score = "partialling out" but will be ignored. "'


In [14]:
table = PrettyTable()
table.field_names = ['Estimator', 'θ_hat', 'Std Error','t','p','2.5%','97.25%']
a = ['OLS']+ np.c_[OLS.params[1], OLS.bse[1], OLS.tvalues[1], OLS.pvalues[1], np.nan, np.nan].reshape(-1).tolist()
table.add_row(a)
a = ['IV-2SLS']+ np.c_[IV2SLS.params[1], IV2SLS.bse[1], IV2SLS.tvalues[1], IV2SLS.pvalues[1], np.nan, np.nan].reshape(-1).tolist()
table.add_row(a)
#a = ['Naive/Single ML (SML)']+ np.array(SML.summary).reshape(-1).tolist()
#table.add_row(a)
#a = ['Orthogonal ML (OML)']+ np.array(OML.summary).reshape(-1).tolist()
#table.add_row(a)
a = ['Orthogonal+Crossfitting ML (DML)']+ np.array(DML.summary).reshape(-1).tolist()
table.add_row(a)
table.float_format = '0.3'
print(table)

+----------------------------------+--------+-----------+--------+-------+--------+--------+
|            Estimator             | θ_hat  | Std Error |   t    |   p   |  2.5%  | 97.25% |
+----------------------------------+--------+-----------+--------+-------+--------+--------+
|               OLS                | -0.131 |   0.018   | -7.122 | 0.000 |  nan   |  nan   |
|             IV-2SLS              | -0.157 |   0.039   | -3.976 | 0.000 |  nan   |  nan   |
| Orthogonal+Crossfitting ML (DML) | -0.155 |   0.064   | -2.414 | 0.016 | -0.280 | -0.029 |
+----------------------------------+--------+-----------+--------+-------+--------+--------+
